<a href="https://colab.research.google.com/github/madhuriawachar1/DSOnMacARM/blob/main/podcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this Demo, we will:

1. Download ggml-whisper which is a really fast version of whisper written in C/C++
2. Convert an audio file with any format to wav (the only format currently supported by ggml-whisper)
3. Convert the transcription to csv and save for model training later

## Download an example video

In [1]:
!wget -O /content/1.mp3 "https://www.dropbox.com/s/9llt4ptcqvr77md/1.mp3?dl=0"

--2023-08-05 07:44:42--  https://www.dropbox.com/s/9llt4ptcqvr77md/1.mp3?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/9llt4ptcqvr77md/1.mp3 [following]
--2023-08-05 07:44:42--  https://www.dropbox.com/s/raw/9llt4ptcqvr77md/1.mp3
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd7a98ceb5bbb92a61a362a6371.dl.dropboxusercontent.com/cd/0/inline/CBPaxdWW0kHX7tZ9G5xDTfyTkdBENayjIlGs9IhhFT0bEidBVcOMNOSfCCxuXeQ1L2qWK2F8OcVFg1T7QVDgM-9u0G_96tv45mm3hWHolMtD0RydWsWkfN3XO6ad2_E5CgSWkuSeex5pKKf3JvcBH5D2/file# [following]
--2023-08-05 07:44:42--  https://ucd7a98ceb5bbb92a61a362a6371.dl.dropboxusercontent.com/cd/0/inline/CBPaxdWW0kHX7tZ9G5xDTfyTkdBENayjIlGs9IhhFT0bEidBVcOMNOSfCCxuXeQ1L2qWK2F8OcVFg1T7QVDgM-9u0G_96tv45mm3hWHolMtD0RydWsWkfN

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
'''import shutil
for i in range(1,25):
  str1=str(i)
  s="/content/drive/MyDrive/"+str1+".mp3"
  d="/content/"+str1+".mp3"
  # Define the source and destination paths
  source_path = s
  destination_path = d

  # Move the file
  shutil.move(source_path, destination_path)'''


## Download ggml-whisper repo and compile

In [2]:
!git clone https://github.com/ggerganov/whisper.cpp.git

Cloning into 'whisper.cpp'...
remote: Enumerating objects: 4091, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 4091 (delta 29), reused 47 (delta 15), pack-reused 4015
Receiving objects: 100% (4091/4091), 7.00 MiB | 10.68 MiB/s, done.
Resolving deltas: 100% (2552/2552), done.


In [3]:
%cd /content/whisper.cpp/models
!bash download-ggml-model.sh base.en
%cd ..
!make

/content/whisper.cpp/models
ggml-base.en.bin    100%[===================>] 141.11M   204MB/s    in 0.7s    
Done! Model 'base.en' saved in 'models/ggml-base.en.bin'
You can now use it like this:

  $ ./main -m models/ggml-base.en.bin -f samples/jfk.wav

/content/whisper.cpp
I whisper.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -DNDEBUG -std=c11   -fPIC -pthread -mavx2 -mfma -mf16c -mavx -msse3
I CXXFLAGS: -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -pthread
I LDFLAGS:  
I CC:       cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:      g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

cc  -I.              -O3 -DNDEBUG -std=c11   -fPIC -pthread -mavx2 -mfma -mf16c -mavx -msse3   -c ggml.c -o ggml.o
g++ -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -pthread -c whisper.cpp -o whisper.o
g++ -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -pthread examples/main/main.cpp examples/common.cpp examples/common-ggml.cpp ggml.o whisper.o -o m

## Run ggml-whisper on sample file

In [ ]:
# Note that main example currently runs only with 16-bit WAV files,
# convert input before running the tool.'
import os

path = "/content/drive/MyDrive/audio_3ai2"
i = 0
print(path)
# Loop through the files in the directory
for file in os.listdir(path):
    if file.endswith(".mp3"):  # Process only MP3 files
        i += 1
        print(file)

        # Convert the MP3 to WAV using FFmpeg
        input_file = os.path.join(path, file)
        output_file = os.path.join(path, f"wav_files/{i}.wav")
        !ffmpeg -i "$input_file" -ar 16000 -ac 1 -c:a pcm_s16le "$output_file"

        # Process the WAV file using the "main" program
        !./main -f "$output_file"



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Redirect and save transcription output and transform to csv

In [6]:
import os
path = "/content/drive/MyDrive/audio_3ai2"
wav_folder = os.path.join(path, f"wav_files")
i=0
for wavf in wav_folder:
  i=i+1
  dest = os.path.join(path, f"transcripts/{i}.txt")
  !./main -f wavf > dest


whisper_init_from_file_no_state: loading model from 'models/ggml-base.en.bin'
whisper_model_load: loading model
whisper_model_load: n_vocab       = 51864
whisper_model_load: n_audio_ctx   = 1500
whisper_model_load: n_audio_state = 512
whisper_model_load: n_audio_head  = 8
whisper_model_load: n_audio_layer = 6
whisper_model_load: n_text_ctx    = 448
whisper_model_load: n_text_state  = 512
whisper_model_load: n_text_head   = 8
whisper_model_load: n_text_layer  = 6
whisper_model_load: n_mels        = 80
whisper_model_load: ftype         = 1
whisper_model_load: qntvr         = 0
whisper_model_load: type          = 2
whisper_model_load: mem required  =  310.00 MB (+    6.00 MB per decoder)
whisper_model_load: adding 1607 extra tokens
whisper_model_load: model ctx     =  140.66 MB
whisper_model_load: model size    =  140.54 MB
whisper_init_state: kv self size  =    5.25 MB
whisper_init_state: kv cross size =   17.58 MB
error: failed to open 'wavf' as WAV file
error: failed to read WAV file '

In [ ]:
# !./main -f '/content/1.wav' > /content/transcription.txt

In [ ]:
import os
import csv

def generate_transcription_csv(folder_path):
    csv_file_path = os.path.join(folder_path, 'transcriptions.csv')

    with open(csv_file_path, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['time_stamp', 'transcription', 'file_name'])  # Writing header

        # Iterate through transcript files in the "transcripts" folder
        transcript_folder = os.path.join(folder_path, 'transcripts')
        for transcript_file in os.listdir(transcript_folder):
            if transcript_file.endswith('.txt'):
                transcript_path = os.path.join(transcript_folder, transcript_file)

                # Extract the base name (without extension) of the transcript file
                file_name = os.path.splitext(transcript_file)[0]

                # Corresponding audio file path
                audio_file_path = os.path.join(folder_path, 'wav_files', f'{file_name}.wav')

                with open(transcript_path, 'r') as txt_file:
                    for line in txt_file:
                        line = line.strip()
                        # Skip empty lines
                        if line == '':
                            continue

                        # Split the line into timestamp and transcription
                        time_stamp, transcription = line.split(']', 1)

                        # Remove the leading '[' from the timestamp
                        time_stamp = time_stamp[1:]

                        # Remove leading and trailing spaces from the transcription
                        transcription = transcription.strip()

                        # Skip lines where the transcription is enclosed in square brackets
                        if not (transcription.startswith('[') and transcription.endswith(']')):
                            # Write the data to the CSV file
                            csv_writer.writerow([time_stamp, transcription, file_name])

    print(f"CSV file has been saved at: {csv_file_path}")


# Replace 'audio_3ai2' with the actual folder path containing your data
generate_transcription_csv('audio_3ai2')


In [ ]:
# import csv

# def generate_transcription_csv(text_file_path):
#     csv_file_path = text_file_path.replace('.txt', '.csv')

#     with open(text_file_path, 'r') as txt_file, open(csv_file_path, 'w') as csv_file:
#         csv_writer = csv.writer(csv_file)
#         csv_writer.writerow(['time_stamp', 'transcription'])  # Writing header

#         for line in txt_file:
#             line = line.strip()
#             # Skip empty lines
#             if line == '':
#                 continue

#             # Split the line into timestamp and transcription
#             time_stamp, transcription = line.split(']', 1)

#             # Remove the leading '[' from the timestamp
#             time_stamp = time_stamp[1:]

#             # Remove leading and trailing spaces from the transcription
#             transcription = transcription.strip()

#             # Skip lines where the transcription is enclosed in square brackets
#             if not (transcription.startswith('[') and transcription.endswith(']')):
#                 # Write the data to the CSV file
#                 csv_writer.writerow([time_stamp, transcription])

#     print(f"CSV file has been saved at: {csv_file_path}")

In [1]:
import os
import csv

def generate_transcriptions_to_csv(folder_path):
    transcript_folder = os.path.join(folder_path, 'transcripts')
    csv_folder = os.path.join(folder_path, 'csv_output')

    # Create the CSV folder if it doesn't exist
    os.makedirs(csv_folder, exist_ok=True)

    # Loop through transcript files in the "transcripts" folder
    for transcript_file in os.listdir(transcript_folder):
        if transcript_file.endswith('.txt'):
            transcript_file_path = os.path.join(transcript_folder, transcript_file)
            csv_file_path = os.path.join(csv_folder, transcript_file.replace('.txt', '.csv'))

            with open(transcript_file_path, 'r') as txt_file, open(csv_file_path, 'w') as csv_file:
                csv_writer = csv.writer(csv_file)
                csv_writer.writerow(['time_stamp', 'transcription'])  # Writing header

                for line in txt_file:
                    line = line.strip()
                    # ... (rest of the processing code)

            print(f"CSV file has been saved at: {csv_file_path}")

# Specify the path to the main folder containing transcripts
main_folder_path = "/content/audio_3ai2"

# Process transcripts and generate CSVs
generate_transcriptions_to_csv(main_folder_path)


FileNotFoundError: ignored

In [ ]:
# generate_transcription_csv("/content/transcription.txt")

CSV file has been saved at: /content/transcription.csv


## Modularize and put everything together

In [ ]:
!wget -O /content/1.mp3 "https://www.dropbox.com/s/9llt4ptcqvr77md/1.mp3?dl=0"

--2023-08-04 20:11:47--  https://www.dropbox.com/s/9llt4ptcqvr77md/1.mp3?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/9llt4ptcqvr77md/1.mp3 [following]
--2023-08-04 20:11:47--  https://www.dropbox.com/s/raw/9llt4ptcqvr77md/1.mp3
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb6a117dfe38066d6a4d6c45724.dl.dropboxusercontent.com/cd/0/inline/CBJj-7fAevwBNMAqWgVMpcFsRLibUMAwNvVVseh2qY3wkeC_YdnKR7hz2KAj9o55AehYAuW1Qk4IVqrSkD7iVvs8EeS6oLUH9_oDi-hK79ZQuXA01ZSRB8vRtQlyS_mfmlKZybWqqkE1Fa_c4oTvZ7I2/file# [following]
--2023-08-04 20:11:48--  https://ucb6a117dfe38066d6a4d6c45724.dl.dropboxusercontent.com/cd/0/inline/CBJj-7fAevwBNMAqWgVMpcFsRLibUMAwNvVVseh2qY3wkeC_YdnKR7hz2KAj9o55AehYAuW1Qk4IVqrSkD7iVvs8EeS6oLUH9_oDi-hK79ZQuXA01ZS

In [ ]:
!git clone https://github.com/ggerganov/whisper.cpp.git

%cd /content/whisper.cpp/models
!bash download-ggml-model.sh base.en
%cd ..
!make

!pip install pydub
!apt install ffmpeg

fatal: destination path 'whisper.cpp' already exists and is not an empty directory.
/content/whisper.cpp/models
Model base.en already exists. Skipping download.
/content/whisper.cpp
I whisper.cpp build info: 
I UNAME_S:  Linux
I UNAME_P:  x86_64
I UNAME_M:  x86_64
I CFLAGS:   -I.              -O3 -DNDEBUG -std=c11   -fPIC -pthread -mavx2 -mfma -mf16c -mavx -msse3
I CXXFLAGS: -I. -I./examples -O3 -DNDEBUG -std=c++11 -fPIC -pthread
I LDFLAGS:  
I CC:       cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
I CXX:      g++ (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0

make: Nothing to be done for 'default'.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
import os
from pydub import AudioSegment
import csv
import subprocess

def convert_to_wav(file_path):
    # Split the file path into base and extension
    base, ext = os.path.splitext(file_path)

    # Only process files that aren't already in wav format
    if ext.lower() != ".wav":
        # Load the audio file
        audio = AudioSegment.from_file(file_path)

        # Set frame rate to 16 kHz
        audio = audio.set_frame_rate(16000)

        # Save as wav
        wav_file_path = base + ".wav"
        audio.export(wav_file_path, format="wav")

        print(f"Saved as {wav_file_path}")
    else:
        print(f"File is already in wav format: {file_path}")

    return wav_file_path

def transcript_wav_totext_using_whisper(wav_file_path):
    base, ext = os.path.splitext(wav_file_path)
    command = f"./main -f '{wav_file_path}' >  {base}.txt"
    print(command)
    process = subprocess.run(command, shell=True, check=True)

    text_file_path = f'{base}.txt'

    return text_file_path

def generate_transcription_csv(text_file_path):
    csv_file_path = text_file_path.replace('.txt', '.csv')

    with open(text_file_path, 'r') as txt_file, open(csv_file_path, 'w') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['time_stamp', 'transcription'])  # Writing header

        for line in txt_file:
            line = line.strip()
            # Skip empty lines
            if line == '':
                continue

            # Split the line into timestamp and transcription
            time_stamp, transcription = line.split(']', 1)

            # Remove the leading '[' from the timestamp
            time_stamp = time_stamp[1:]

            # Remove leading and trailing spaces from the transcription
            transcription = transcription.strip()

            # Skip lines where the transcription is enclosed in square brackets
            if not (transcription.startswith('[') and transcription.endswith(']')):
                # Write the data to the CSV file
                csv_writer.writerow([time_stamp, transcription])

    print(f"CSV file has been saved at: {csv_file_path}")

def process_folder(folder_path):
    # Iterate through audio files in the "wav_files" folder
    wav_folder = os.path.join(folder_path, 'wav_files')
    for wav_file in os.listdir(wav_folder):
        if wav_file.endswith('.wav'):
            wav_file_path = os.path.join(wav_folder, wav_file)
            text_file_path = transcript_wav_totext_using_whisper(wav_file_path)
            generate_transcription_csv(text_file_path)


# Specify the main folder path
main_folder_path = "audio_3ai2"

# Call the process_folder function to process all files in the folder
process_folder(main_folder_path)


In [ ]:
# begin('/content/1.mp3')

Saved as /content/1.wav
./main -f '/content/1.wav' >  /content/1.txt
CSV file has been saved at: /content/1.csv
DONE


## Add DOC_ID Column

In [ ]:
import os
import pandas as pd

def add_ids_to_csv_files(folder_path):
    # Iterate through CSV files in the folder
    csv_folder = os.path.join(folder_path, 'csv_files')
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            csv_file_path = os.path.join(csv_folder, csv_file)

            # Read the CSV file
            df = pd.read_csv(csv_file_path)

            # Add the new column with values from 0 to n-1
            df['DOC_ID'] = range(len(df))

            # Save the updated DataFrame back to the CSV file
            df.to_csv(csv_file_path, index=False)

            print(f"Updated {csv_file_path} with IDs.")


# Specify the main folder path
main_folder_path = "audio_3ai2"

# Call the add_ids_to_csv_files function to process all CSV files in the folder
add_ids_to_csv_files(main_folder_path)


In [ ]:
# import pandas as pd

# # Read the CSV file
# df = pd.read_csv('/content/1.csv')

# # Add the new column with values from 0 to n-1
# df['DOC_ID'] = range(len(df))

# # Save the updated DataFrame back to the CSV file
# df.to_csv('/content/1.csv', index=False)

In [ ]:
!pip3 install thirdai --upgrade
!pip3 install thirdai[neural_db]
!pip3 install langchain --upgrade
!pip3 install openai --upgrade
!pip3 install paper-qa --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 85.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 117.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 67.3 MB/s eta 0:00:

In [ ]:
from thirdai import licensing, neural_db as ndb
licensing.deactivate()
licensing.activate("1FB7DD-CAC3EC-832A67-84208D-C4E39E-V3")

In [ ]:
db = ndb.NeuralDB(user_id="root")

In [ ]:
import os
import pandas as pd
from pandas_gbq import ndb

def process_csv_files(folder_path):
    insertable_docs = []

    # Iterate through CSV files in the folder
    csv_folder = os.path.join(folder_path, 'csv_files')
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            csv_file_path = os.path.join(csv_folder, csv_file)

            # Read the CSV file
            df = pd.read_csv(csv_file_path)

            # Create a CSV document
            csv_doc = ndb.CSV(
                path=csv_file_path,
                id_column="DOC_ID",
                strong_columns=["transcription"],
                weak_columns=["time_stamp"],
                reference_columns=["time_stamp"])

            insertable_docs.append(csv_doc)

    return insertable_docs


# Specify the main folder path
main_folder_path = "audio_3ai2"

# Call the process_csv_files function to process all CSV files in the folder
insertable_docs = process_csv_files(main_folder_path)

# Now you can use the insertable_docs list for further processing


In [ ]:
# insertable_docs = []
# csv_files = ['/content/1.csv']

# for file in csv_files:
#     csv_doc = ndb.CSV(
#         path=file,
#         id_column="DOC_ID",
#         strong_columns=["transcription"],
#         weak_columns=["time_stamp"],
#         reference_columns=["time_stamp"])
#     #
#     insertable_docs.append(csv_doc)

In [ ]:
source_ids = db.insert(insertable_docs, train=True)

In [ ]:
# source_ids = db.insert(insertable_docs, train=True)

loaded data | source 'Documents:
1.csv' | vectors 278 | batches 1 | time 0s | complete

train | epoch 0 | train_steps 1 | train_hash_precision@5=0.018705  | train_batches 1 | time 3s

train | epoch 1 | train_steps 2 | train_hash_precision@5=0.551799  | train_batches 1 | time 0s

train | epoch 2 | train_steps 3 | train_hash_precision@5=0.361151  | train_batches 1 | time 1s

train | epoch 3 | train_steps 4 | train_hash_precision@5=0.295683  | train_batches 1 | time 1s

train | epoch 4 | train_steps 5 | train_hash_precision@5=0.328058  | train_batches 1 | time 1s

train | epoch 5 | train_steps 6 | train_hash_precision@5=0.397122  | train_batches 1 | time 0s

train | epoch 6 | train_steps 7 | train_hash_precision@5=0.523741  | train_batches 1 | time 0s

train | epoch 7 | train_steps 8 | train_hash_precision@5=0.666906  | train_batches 1 | time 0s

train | epoch 8 | train_steps 9 | train_hash_precision@5=0.802878  | train_batches 1 | time 0s

train | epoch 9 | train_steps 10 | train_hash_pr

In [ ]:
search_results = db.search(
    query="what is Starkel average for January is 31 degrees",
    top_k=2,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for result in search_results:
    print(result.text)
    print(result.context(radius=1))
    print(result.source)
    print(result.metadata)
    print('************')

00:06:26.000 --> 00:06:33.000
/content/1.csv
{'time_stamp': '00:06:26.000 --> 00:06:33.000', 'transcription': "Starkel average for January's is 31 degrees, last month was 39.5 degrees.", 'DOC_ID': 58}
************
00:06:17.000 --> 00:06:26.000
/content/1.csv
{'time_stamp': '00:06:17.000 --> 00:06:26.000', 'transcription': 'Just in the last two days, we got the new temperature records in January. This is just for the United States of America.', 'DOC_ID': 57}
************
